# Análise de reviews de produtos usando NLP

O presente trabalho pretende analisar os textos de revisão de produtos da base de dados obtida na plataforma Kaggle. Para isso serão usadas bibliotecas Python de processamento de linguagem natural já conhecidas como Spacy, Sci-kit Learn e.

https://www.kaggle.com/olistbr/brazilian-ecommerce

Objetivos:
- Encontrar caracaterísticas que distinguam textos de reviews com notas altas e com notas baixas.
- Analisar textos com análise de sentimentos e identificar textos muito negativos, mas que possuam notas altas.

In [131]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from nltk import corpus
import nltk
import re
from unicodedata import normalize
import itertools as it
import numpy as np

In [132]:
# Definindo a máquina virtual Java para o lematizador e pos-tagger de Stanford
from py4j.java_gateway import JavaGateway

gg = JavaGateway.launch_gateway(classpath="./java/lemmatizerpt.jar;./java/py4j0.10.8.1.jar;./java/stanford-postagger.jar;./java/lemport-0.9.7.jar")


In [133]:
e_set = pd.read_csv("./brazilian-ecommerce/olist_public_dataset_v2.csv")
e_set.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01.000000,2017-02-01 02:41:21.549551,2017-03-15 00:00:00.000000,2017-02-06 11:04:24.154259,...,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07 00:00:00.000000,2017-02-09 02:37:37+00:00
1,59af46052a799e80e2f0c665c587731d,delivered,15.00,15.10,1,1,2017-09-09 19:52:54.000000,2017-09-10 20:03:31.535281,2017-10-02 00:00:00.000000,2017-09-13 20:17:41.296915,...,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,NaN,entrega em 2 dias produto c boa qualidade otim...,2017-09-14 00:00:00.000000,2017-09-15 03:43:47+00:00
2,a3e6136894621db402a772c6bc72a12a,delivered,238.90,18.00,1,1,2017-01-30 17:00:09.000000,2017-01-30 17:31:25.438253,2017-03-07 00:00:00.000000,2017-02-06 15:43:04.758566,...,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,NaN,produto veio antes do prazo informado muito ob...,2017-02-07 00:00:00.000000,2017-02-10 14:18:53+00:00
3,b675ea5a618922f6e679e30531b8957b,delivered,29.99,18.23,1,1,2018-03-11 18:18:36.000000,2018-03-11 18:30:37.931962,2018-04-03 00:00:00.000000,2018-04-03 20:36:43.778451,...,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,NaN,NaN,2018-04-04 00:00:00.000000,2018-04-05 02:52:31+00:00
4,195a8be6794c487fe6cfbb97b7c61902,delivered,295.99,47.65,1,1,2017-04-20 08:01:08.000000,2017-04-25 08:05:40.405383,2017-05-24 00:00:00.000000,2017-05-04 18:47:45.721758,...,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,NaN,NaN,2017-05-05 00:00:00.000000,2017-05-08 15:20:18+00:00


# Padronizando o DataFrame

Retiramos todas as linhas que não possuem reviews para analisar somente os textos.

In [134]:
e_work_set = e_set[['order_products_value','review_score', 'review_comment_message']].copy()
e_work_set = e_work_set.dropna().reset_index(drop = True)
e_work_set.head()

,order_products_value,review_score,review_comment_message
0,15.0,5,entrega em 2 dias produto c boa qualidade otim...
1,238.9,5,produto veio antes do prazo informado muito ob...
2,160.0,4,chegou bem antes do prazo ótimo vendedor
3,25.0,3,gostei do produto
4,187.0,5,o produto veio certinho muito bem embalado e b...


## Pre-processamento

## Remoção de acentuação 

In [135]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')



## Remoção de pontuação

Será feita a remoção de acentuação usando uma expressão regular simples.

In [136]:
pattern = re.compile('[^a-zA-Z\d\s]')
#print(re.sub(pattern,'','Amanhã, iremos a casa!'))

## Remoção de Stopwords

As stopwords, palavras que não possuem sentido significativo para a sentença são retiradas através da corpora disponibilizada pela bibliteca NLTK.

In [137]:
stop_words = set(corpus.stopwords.words('portuguese'))

stop_words_norm = []
for i in stop_words:
    word = remover_acentos(i)
    stop_words_norm.append(word)
    
stop_words_norm.remove('nao')

## Lematização

A lematização é a redução da palavra a sua forma mais básica, ou seja, substanttivos na forma masculina e no singular, verbos no infinitivo.

O procedimento foi feito adptando o lemmatizador stanford, feito em Java, usando a biblioteca Py4J que cria uma interface com Java para Python

## Realização do pré processamento

Ao final do pré-processamento obteremos um texto simplificado, mas que padroniza a estrutura dos comentários e facilita na nossa análise.

In [138]:
gg = JavaGateway.launch_gateway(classpath="./java/lemmatizerpt.jar;./java/py4j0.10.8.1.jar;./java/stanford-postagger.jar;./java/lemport-0.9.7.jar")
lemmatizer = gg.jvm.lemmatizer.LemmatizerInterface()    

for index,row in e_work_set.iterrows():
    
    # Retiramos a pontuação usando Regex.
    lista_tks = []
    for tk in e_work_set['review_comment_message'][index].split():
        lista_tks.append(re.sub(pattern,'',tk))
    tk_reviews = lista_tks
    
    # Reduz a palavra ao seu lema.
    
    lemmatizer.processLemmas(' '.join(tk_reviews))
    
    list_lemmas = [i for i in lemmatizer.getLemmas()]
    list_tags = [j for j in lemmatizer.getPosTags()]    

    # Removemos os acentos inicialmente.
    tk_reviews = remover_acentos(' '.join(list_lemmas))
       
    # Retira Stopwords.
    tk_reviews = [w for w in tk_reviews.split() if not w in stop_words_norm]
    
    e_work_set.loc[index,'review_tratada'] = ' '.join(tk_reviews)
    e_work_set.loc[index,'pos_tags'] = ' '.join(list_tags)
    
    

In [139]:
e_work_set.to_csv('review_lematizada.csv',encoding = 'utf-8',index = False)

In [140]:
e_work_set.head()

,order_products_value,review_score,review_comment_message,review_tratada,pos_tags
0,15.0,5,entrega em 2 dias produto c boa qualidade otim...,entrega 2 dia produto c bom qualidade otimo cu...,n prp num n n adv adj n adj n n
1,238.9,5,produto veio antes do prazo informado muito ob...,produto vir antes prazo informar obrigar agili...,n v-fin prp adj n v-pcp adv v-pcp n adv
2,160.0,4,chegou bem antes do prazo ótimo vendedor,chegar bem antes prazo timo vendedor,v-fin adv adv adj n adj n
3,25.0,3,gostei do produto,gostar produto,v-fin adj n
4,187.0,5,o produto veio certinho muito bem embalado e b...,produto vir certo bem embalar bem antes prazo ...,art n v-fin adj adv adv v-pcp conj-c adv adv a...


# Extraindo Features das reviews tokenizadas

Agora vamos extrair algumas características das reviews para analisarmos separadamente os textos de reviews para cada pontuação.

In [141]:
e_set_rev_1 = e_work_set[e_work_set['review_score'] == 1].reset_index(drop = True)
e_set_rev_2 = e_work_set[e_work_set['review_score'] == 2].reset_index(drop = True)
e_set_rev_3 = e_work_set[e_work_set['review_score'] == 3].reset_index(drop = True)
e_set_rev_4 = e_work_set[e_work_set['review_score'] == 4].reset_index(drop = True)
e_set_rev_5 = e_work_set[e_work_set['review_score'] == 5].reset_index(drop = True)
e_set_rev_1.head()

,order_products_value,review_score,review_comment_message,review_tratada,pos_tags
0,134.90,1,o correio ainda não fez a entrega to furioso,correio ainda fazer entrega to furioso,art n adv pron-pers v-fin art n adj adj
1,329.99,1,não recebi o meu produto e nem fui avisada da ...,receber produto ser avisar faltar item loja en...,pron-pers v-fin art pron-det n conj-c adv v-fi...
2,251.90,1,comprei um produto e recebi outro e inferior g...,comprar produto receber outro inferior gostar ...,v-fin art n conj-c v-fin pron-det conj-c adj v...
3,979.00,1,não comprem só vai dar dor de cabeça o produto...,comprar s ir dar dor cabea produto nunca chegar,pron-pers v-fin pron-pers v-fin v-inf n prp n ...
4,62.70,1,comprei 2 produtos iguais porém em cores difer...,comprar 2 produto igual porm cor diferente che...,v-fin num n adj n prp n adj v-fin prp pron-det...


## Análise Exploratória

Será mostrado, como exemplo, o código para extração de cada feature nas reviews de nota 1. Em seguida mostraremos apenas os resultados obtidos para as reviews agrupadas por nota.

## Media de Tokens

Com o propósito de analisar o tamanho do texto escrito pelos consumidores ao avaliar um produto, foi feita uma média dos tokens para cada nota.

In [168]:
def media_tokens(df):
    vectorizer = CountVectorizer(encoding = 'utf_8')
    reviews = []

    for row in df.itertuples():
        reviews.append(row[3])

    tokenized_reviews_all = vectorizer.fit_transform(reviews)
    nr_medio_de_tokens_1 = tokenized_reviews_all.toarray().sum()/len(e_set_rev_1)
    return nr_medio_de_tokens_1


In [170]:
print('Média de Tokens para reviews de nota 1: %.2f' % media_tokens(e_set_rev_1))
print('Média de Tokens para reviews de nota 2: %.2f' % media_tokens(e_set_rev_2))
print('Média de Tokens para reviews de nota 3: %.2f' % media_tokens(e_set_rev_3))
print('Média de Tokens para reviews de nota 4: %.2f' % media_tokens(e_set_rev_4))
print('Média de Tokens para reviews de nota 5: %.2f' % media_tokens(e_set_rev_5))

Média de Tokens para reviews de nota 1: 15.83
Média de Tokens para reviews de nota 2: 3.80
Média de Tokens para reviews de nota 3: 5.67
Média de Tokens para reviews de nota 4: 6.20
Média de Tokens para reviews de nota 5: 17.05


## Análise de Sentimento

A análise de sentimento proposta é feita de maneira bem simples. Baseado em uma lista de termos positivos e uma lista de termos negativos calculamos a porcentagem de palavras positivas e negativas no total. Palavras que não estão classificadas em positivas e negativas serão consideradas neutras.

As listas são provenientes do site Kaggle.
https://www.kaggle.com/rtatman/sentiment-lexicons-for-81-languages

### Importando léxico de palavras negativas

In [143]:
palavras_negativas = []

with open('./sentiment-lexicons/negative_words_pt.txt','r',encoding = 'utf_8') as f:
    for line in f.readlines():
        palavras_negativas.append(line)


lista_negativas = []
for i in palavras_negativas:
    palavra = i.replace('\n', '')
    lista_negativas.append(remover_acentos(palavra))   

lista_negativas = set(lista_negativas)


### Importando léxico de palavras positivas

In [144]:
palavras_positivas = []

with open('./sentiment-lexicons/positive_words_pt.txt','r',encoding = 'utf_8') as f:
    for line in f.readlines():
        palavras_positivas.append(line)


lista_positivas = []
for i in palavras_positivas:
    palavra = i.replace('\n', '')
    lista_positivas.append(remover_acentos(palavra))   

lista_positivas = set(lista_positivas)

### Definindo função para contagem de tokens positivos e negativos

In [158]:
# Recebe uma string e comparar as palavras que estão no léxico positivo e negativo

def sent_count(text):
    pos_count = 0
    neg_count = 0
    for token in text.split():
        if token in lista_positivas:
            pos_count += 1
            
        elif token in lista_negativas:
            neg_count += 1
            
    return((pos_count,neg_count))

In [159]:
for index,row in e_work_set.iterrows():
    x = e_work_set['review_tratada'][index]

    if x == '':
        continue
    sentimentos = sent_count(x)
    e_work_set.loc[index,'nr_tk_positivos'] = sentimentos[0]
    e_work_set.loc[index,'porcent_tk_positivos'] = sentimentos[0]/len(x.split())
    e_work_set.loc[index,'nr_tk_negativos'] = sentimentos[1]
    e_work_set.loc[index,'porcent_tk_negativos'] = sentimentos[1]/len(x.split())
    
    if e_work_set['porcent_tk_positivos'][index] == e_work_set['porcent_tk_negativos'][index]:
        e_work_set.loc[index,'sentimento'] = 'Neutro'
        
    elif e_work_set['porcent_tk_positivos'][index] > e_work_set['porcent_tk_negativos'][index]:
        e_work_set.loc[index,'sentimento'] = 'Positivo'
        
    else:
        e_work_set.loc[index,'sentimento'] = 'Negativo'



Após a contagem de termos positivos e negativos as reviews foram classificadas em sentimento negativo e positivo beasedo apenas na porcentagem de termos positivos e termos negativos.

## Separando os dataframes por review 

In [160]:
e_set_rev_1 = e_work_set[e_work_set['review_score'] == 1].reset_index(drop = True)
e_set_rev_2 = e_work_set[e_work_set['review_score'] == 2].reset_index(drop = True)
e_set_rev_3 = e_work_set[e_work_set['review_score'] == 3].reset_index(drop = True)
e_set_rev_4 = e_work_set[e_work_set['review_score'] == 4].reset_index(drop = True)
e_set_rev_5 = e_work_set[e_work_set['review_score'] == 5].reset_index(drop = True)
e_set_rev_1.head()

,order_products_value,review_score,review_comment_message,review_tratada,pos_tags,nr_tk_positivos,porcent_tk_positivos,nr_tk_negativos,porcent_tk_negativos,sentimento
0,134.90,1,o correio ainda não fez a entrega to furioso,correio ainda fazer entrega to furioso,art n adv pron-pers v-fin art n adj adj,0.0,0.000000,1.0,0.166667,Negativo
1,329.99,1,não recebi o meu produto e nem fui avisada da ...,receber produto ser avisar faltar item loja en...,pron-pers v-fin art pron-det n conj-c adv v-fi...,3.0,0.150000,2.0,0.100000,Positivo
2,251.90,1,comprei um produto e recebi outro e inferior g...,comprar produto receber outro inferior gostar ...,v-fin art n conj-c v-fin pron-det conj-c adj v...,1.0,0.071429,1.0,0.071429,Neutro
3,979.00,1,não comprem só vai dar dor de cabeça o produto...,comprar s ir dar dor cabea produto nunca chegar,pron-pers v-fin pron-pers v-fin v-inf n prp n ...,1.0,0.111111,1.0,0.111111,Neutro
4,62.70,1,comprei 2 produtos iguais porém em cores difer...,comprar 2 produto igual porm cor diferente che...,v-fin num n adj n prp n adj v-fin prp pron-det...,0.0,0.000000,0.0,0.000000,Neutro


In [161]:
df = pd.DataFrame()
df['sent_1'] = e_set_rev_1['sentimento'].value_counts()
df['sent_2'] = e_set_rev_2['sentimento'].value_counts()
df['sent_3'] = e_set_rev_3['sentimento'].value_counts()
df['sent_4'] = e_set_rev_4['sentimento'].value_counts()
df['sent_5'] = e_set_rev_5['sentimento'].value_counts()

df.head()

,sent_1,sent_2,sent_3,sent_4,sent_5
Neutro,3226,741,1119,1693,5194
Negativo,3223,766,1053,721,875
Positivo,2851,846,1794,3687,14686


In [162]:
eval_df_1 = e_set_rev_1[e_set_rev_1['sentimento'] == 'Positivo']
# eval_df_1.head(len(eval_df_1))


,order_products_value,review_score,review_comment_message,review_tratada,pos_tags,nr_tk_positivos,porcent_tk_positivos,nr_tk_negativos,porcent_tk_negativos,sentimento
1,329.99,1,não recebi o meu produto e nem fui avisada da ...,receber produto ser avisar faltar item loja en...,pron-pers v-fin art pron-det n conj-c adv v-fi...,3.0,0.150000,2.0,0.100000,Positivo
12,113.00,1,não foi entregue o produto que comprei enviara...,ser entregar produto comprar enviar outro prod...,pron-pers v-fin v-pcp art n pron-indp v-fin v-...,2.0,0.100000,1.0,0.050000,Positivo
13,89.90,1,devolvi as toalhas porque não foi o que espera...,devolver toalha porque ser esperar relao quali...,v-fin art n conj-s pron-pers v-fin pron-det pr...,1.0,0.055556,0.0,0.000000,Positivo
16,40.00,1,bom dia o lustre anunciado era azul claro foi ...,bom dia lustre anunciar ser azul claro ser ent...,adj n art n v-pcp v-fin adj adv v-fin v-pcp ad...,5.0,0.454545,2.0,0.181818,Positivo
22,69.80,1,só recebi um item o pedido foi feito e peguei ...,s receber item pedido ser fazer peguei 2 item ...,n v-fin art n art n v-fin v-pcp conj-c adj prp...,2.0,0.125000,0.0,0.000000,Positivo
25,371.70,1,na foto do site consta o produto com 120 cápsu...,foto site constar produto 120 cpsula pote rece...,pron-pers n adj n v-fin art n prp num n art n ...,1.0,0.058824,0.0,0.000000,Positivo
27,65.80,1,nada a opinar que não seja o fato de ter pago ...,nada opinar ser fato ter pagar ter receber pro...,pron-indp pron-det n pron-indp pron-pers v-fin...,3.0,0.150000,1.0,0.050000,Positivo
37,46.90,1,não recomendo essa loja pra ninguém comprei o ...,recomendar loja pra ningum comprar presente fi...,pron-pers v-fin pron-det n prp n v-fin art n a...,2.0,0.080000,1.0,0.040000,Positivo
42,19.90,1,não gostei do produto não funciona como aparec...,gostar produto funcionar aparecer propaganda,pron-pers v-fin adj n pron-pers v-fin adv v-fi...,3.0,0.600000,1.0,0.200000,Positivo
43,239.99,1,olá queria informar que o produto não foi entr...,ol querer informar produto ser entregar,adj v-fin v-inf conj-s art n pron-pers v-fin v...,1.0,0.166667,0.0,0.000000,Positivo


In [150]:
eval_df_5 = e_set_rev_5[e_set_rev_5['sentimento'] == 'Negativo']
eval_df_5.head(len(eval_df_5))

,order_products_value,review_score,review_comment_message,review_tratada,pos_tags,nr_tk_positivos,porcent_tk_positivos,nr_tk_negativos,porcent_tk_negativos,sentimento
17,188.00,5,já fiz várias compras pelo stark nuca tive pro...,j fazer vria compra stark nuca ter problema,n v-fin adj n adj n adj v-fin n,0.0,0.000000,1.0,0.125000,Negativo
35,99.00,5,o produto comprado embora diga que é específic...,produto comprar embora dizer especfiquir celul...,art n v-pcp conj-s v-fin conj-s v-inf prp art ...,1.0,0.058824,2.0,0.117647,Negativo
41,21.90,5,nao abri ainda pq nao vou usar agora,naar abri ainda pq nao ir usar agora,v-fin n adv v-fin n v-fin v-inf adv,0.0,0.000000,1.0,0.125000,Negativo
53,422.00,5,produto conforme anuncio chegou antes do prazo,produto conforme anunciar chegar antes prazo,n adv v-fin v-fin adv adj n,0.0,0.000000,1.0,0.166667,Negativo
79,79.90,5,o produto chegou até antes do prazo estou sati...,produto chegar at antes prazo estar satisfeiti...,art n v-fin n adv adj n v-fin adv v-pcp v-fin ...,0.0,0.000000,1.0,0.062500,Negativo
94,119.80,5,olá meu produto nao recebi ainda mais sempre r...,ol produto nao receber ainda sempre receber tudo,adj pron-det n n v-fin adv adv adv v-fin pron-...,0.0,0.000000,1.0,0.125000,Negativo
213,73.00,5,a caixa é linda as divisórias internas podem s...,caixa lindar divisria interno poder ser retira...,art n v-pcp art n adj v-fin v-inf v-pcp n v-fi...,2.0,0.086957,3.0,0.130435,Negativo
268,49.90,5,incrível que pela primeira vez que demora pra ...,incrvel pelar primeira vez demorar pra entrega...,n pron-indp v-fin adj n pron-indp v-fin prp n ...,0.0,0.000000,1.0,0.066667,Negativo
273,54.00,5,produto conforme anunciado,produto conforme anunciar,n adv v-pcp,0.0,0.000000,1.0,0.333333,Negativo
277,139.00,5,ótimo produto empresa cumpre o anunciado pneus...,timo produto empresa cumprir anunciar pneu mag...,adj n n v-fin art v-pcp n v-fin num num conj-c...,0.0,0.000000,1.0,0.090909,Negativo


Analisando as reviews consideradas com sentimento positivo e com nota baixa vemos que algumas reviews analisam o histórico do desempenho da loja como positivo, mas relatam problemas atuais com o pedido atual.

Além disso, vemos algumas palavras que são podem ser consideradas positivas, mas que são alteradas pelo contexto, por exemplo "não recomendo". 

In [164]:
eval_df_1['review_comment_message'][9246]

'o targaryen já foi uma boa loja estou a duas semanas tentando falar com eles sobre um produto fun kitchen que deu defeito para eu procurar uma autorizada e não consigo eu não recomendo essa loja'

Vê-se uma situação semelhante nas reviews de nota alta e sentimento predominantemente negativo. Há o uso de palavras que são consideradas negativas como "problema", mas tem seu sentido alterado pelo contexto.

Há também o uso de advérbios de intensidade, que são contados como positivos, mas que podem amplificar um sentimento negativo

In [165]:
eval_df_5['review_comment_message'][20343]

'compro a muito tempo nunca tive problemas hoje melhorou mais ainda parabéns vocês são 10'

### Encontrar as n palavras mais frequentes

Foi feita uma função para se obter as palavras mais frequentes em um conjunto de reviews. No caso as reviews estão agrupadas pela nota dada ao produto.

In [151]:
def get_top_n_words(dataframe, n=None):
    
    # Encontra as palavras de maior frequência em um corpus e retorna como um par palavra, nº de ocorrências
    
    corpus = []
    for linha in dataframe.itertuples():
        corpus.append(linha[4])
    
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

    return words_freq[:n]


In [152]:
top_w_1 = pd.DataFrame(get_top_n_words(e_set_rev_1,n = 10),columns = ['Palavra','Frequência'])
top_w_1

,Palavra,Frequência
0,produto,5511
1,receber,3576
2,ser,3170
3,comprar,2312
4,entregar,2199
5,ter,1529
6,chegar,1450
7,vir,1423
8,estar,1255
9,ainda,1232


In [153]:
top_w_2 = pd.DataFrame(get_top_n_words(e_set_rev_2,n = 10),columns = ['Palavra','Frequência'])
top_w_2

,Palavra,Frequência
0,produto,1286
1,ser,813
2,receber,672
3,vir,495
4,entregar,484
5,comprar,470
6,ter,386
7,chegar,367
8,estar,297
9,ainda,254


In [154]:
top_w_3 = pd.DataFrame(get_top_n_words(e_set_rev_3,n = 10),columns = ['Palavra','Frequência'])
top_w_3

,Palavra,Frequência
0,produto,1951
1,ser,1221
2,vir,772
3,receber,770
4,ter,696
5,bom,684
6,entregar,652
7,chegar,588
8,comprar,509
9,prazo,502


In [155]:
top_w_4 = pd.DataFrame(get_top_n_words(e_set_rev_4,n = 10),columns = ['Palavra','Frequência'])
top_w_4

,Palavra,Frequência
0,produto,2681
1,bom,1518
2,prazo,1394
3,ser,1147
4,entregar,1056
5,chegar,940
6,antes,762
7,recomendar,596
8,entrega,587
9,vir,581


In [156]:
top_w_5 = pd.DataFrame(get_top_n_words(e_set_rev_5,n = 10),columns = ['Palavra','Frequência'])
top_w_5

,Palavra,Frequência
0,produto,8654
1,prazo,5680
2,antes,4481
3,entregar,4019
4,bom,3999
5,recomendar,3577
6,chegar,3505
7,bem,2274
8,ser,2234
9,entrega,1958


Analisando as palavras mais frequêntes encontradas em cada agrupamento de reviews vemos que há nos reviews de notas mais altos palavras como "bom", "recomendar","bem" e "antes" estão mais presentes.

Em contrapartida, vemos que o que diferencia as reviews de nota mais baixa é apenas a palavra "ainda", que provavelmente está associada ao fato de o produto não ter sido entregue.

As palavras comuns a todas as reviews referem-se a palavras frequêntes no contexto analisado. Palavras como "produto","prazo","entregar" estaram provavelmente em todas as reviews e não possuem valor de sentimento atrelados a elas.

# Conclusão

## Sobre as reviews

Vemos dados interessantes retirados das reviews. Apesar da simplicidade do método, vemos que o número de reviews com sentimento positivo era muito maior para as reviews de nota 5, além de crescerem a partir das reviews de nota 1.

Analisando as médias de tokens para cada grupo de reviews, vemos que os grupos com mais tokens são as reviews de nota 1 e nota 5. Isso nos leva a teorizar que as pessoas escrevem mais quando seus textos possuem sentimentos mais intensos, sejam positivos ou negativos.

## Sobre os métodos utilizados

Ao final desse trabalho é importante destacar as limitações dos métodos da NLP. Apesar de apresentar resultados satisfatórios, vemos que dificilmente uma análise de sentimento de metodologia tão simples poderia dizer sobre o teor da mensagem. Um exemplo disso é que, não é possível mostrar o uso de alongamentos em uma palavra para demonstrar intensidade de um sentimento, por exemplo, "adoreeeeei".

Para uma análise de sentimentos com maior precisão seria recomendável a utilização de serviços como a inteligência artificial Watson da IBM.

O lematizador também se mostrou um problema. Várias vezes ele considerou as últimas letras de alguma palavra como algum morfema de flexão verbal e criou um verbo inexistente. Isso acarretou uma perda de precisão em todos os outros processos de pré porocessamento utilizados.